# 0.0. Introducao

## 0.1. Bibliotecas

In [143]:
import numpy             as np
import pandas            as pd
import streamlit         as st
import plotly.express    as px
import seaborn           as sns
import matplotlib.pyplot as plt

from datetime import datetime

import warnings

st.set_page_config( layout= 'wide' )
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.options.display.max_columns = None
warnings.filterwarnings( 'ignore' )

## 0.2. Funcoes auxiliares

## 0.3. Carregamento dos dados

In [2]:
df1 = pd.read_csv( 'house_rocket.csv')

# 1.0 Descrição dos Dados

## 1.1. Dinmensao dos dados

In [3]:
print('Número de linhas: {}.'.format( df1.shape[0] ) )
print('Número de colunas: {}.'.format( df1.shape[1] ) )

Número de linhas: 21613.
Número de colunas: 21.


## 1.2.Tipos de dados

In [4]:
df1['date'] = pd.to_datetime( df1['date'] ).dt.strftime( '%Y-%m-%d' )
df1.dtypes

id                 int64
date              object
price            float64
bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront         int64
view               int64
condition          int64
grade              int64
sqft_above         int64
sqft_basement      int64
yr_built           int64
yr_renovated       int64
zipcode            int64
lat              float64
long             float64
sqft_living15      int64
sqft_lot15         int64
dtype: object

## 1.3. Check NA

In [5]:
df1.isna().sum()

id               0
date             0
price            0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
zipcode          0
lat              0
long             0
sqft_living15    0
sqft_lot15       0
dtype: int64

## 1.4. Mudanca tipos de dados

In [6]:
df1['bathrooms'] = round( df1['bathrooms'] ).astype( 'int64' )

## 1.5. Estatística descritiva

In [7]:
num_atributos = df1.select_dtypes( include = ['int64', 'float64'] )

### 1.5.1. Atributos numericos

In [8]:
# medidas de tendencia central - mean, median
tc1 = pd.DataFrame( num_atributos.apply( np.mean ) ).T
tc2 = pd.DataFrame( num_atributos.apply( np.median ) ).T

# medidas de dispersao - min, max, std, range, knew, kurtosis
d1 = pd.DataFrame( num_atributos.apply( min ) ).T
d2 = pd.DataFrame( num_atributos.apply( max ) ).T
d3 = pd.DataFrame( num_atributos.apply( np.std ) ).T
d4 = pd.DataFrame( num_atributos.apply( lambda x: x.max() - x.min() ) ).T
d5 = pd.DataFrame( num_atributos.apply( lambda x: x.skew() ) ).T
d6 = pd.DataFrame( num_atributos.apply( lambda x: x.kurtosis() ) ).T

m = pd.concat( [d1, d2, d4, tc1, tc2, d3, d5, d6] ).T.reset_index()
m.columns = ['atributos', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']
m

,atributos,min,max,range,mean,median,std,skew,kurtosis
0,id,1000102.00,9900000190.00,9899000088.00,4580301520.86,3904930410.00,2876499023.43,0.24,-1.26
1,price,75000.00,7700000.00,7625000.00,540088.14,450000.00,367118.70,4.02,34.59
2,bedrooms,0.00,33.00,33.00,3.37,3.00,0.93,1.97,49.06
3,bathrooms,0.00,8.00,8.00,2.06,2.00,0.76,1.08,2.75
4,sqft_living,290.00,13540.00,13250.00,2079.90,1910.00,918.42,1.47,5.24
5,sqft_lot,520.00,1651359.00,1650839.00,15106.97,7618.00,41419.55,13.06,285.08
6,floors,1.00,3.50,2.50,1.49,1.50,0.54,0.62,-0.48
7,waterfront,0.00,1.00,1.00,0.01,0.00,0.09,11.39,127.63
8,view,0.00,4.00,4.00,0.23,0.00,0.77,3.40,10.89
9,condition,1.00,5.00,4.00,3.41,3.00,0.65,1.03,0.53


## 1.6. Exclusao de outliers

In [9]:
df1 = df1[df1['bedrooms'] != 33]

# 2.0. Feature Engeneering

In [10]:
df2 = df1.copy()

## 2.1. Criação de Hipoteses

- H1) Imóveis que possuem vista para água, são 30% mais caros, na média.
- H2) Imóveis com data de construção menor que 1955, são 50% mais baratos, na média.
- H3) Imóveis sem porão possuem sqft_lot 50% maiores do que imóveis com porão.
- H4) O crescimento do preço dos imóveis YoY ( Year over Year ) é de 10%.
- H5) Imóveis reformados com 3 banheiros são, em média, 30% mais caros do que imóveis não reformados com 3 banheiros.
- H6) Imóveis vendidos no verão são mais caros.
- H7) Imóveis construídos nos 2000 são 20% mais caros do que imóveis construídos nos anos de 1980.

## 2.2. Feature Engeneering

In [161]:
# construção antes e depois de 1955
df2['before_after_1955'] = df1['yr_built'].apply( lambda x: 'before' if x <= 1955 else 'after')

# feature year

df2['year'] = pd.to_datetime(df2['date']).dt.strftime( '%Y')

# feature month_day
df2['month_day'] = pd.to_datetime(df2['date']).dt.strftime( '%m-%d')

#feature week_of_year

df2['week_of_year'] = pd.to_datetime( df2['date'] ).dt.strftime( '%Y-%U' ) 

# feature is_basement
df2['is_basement'] = df2['sqft_basement'].apply( lambda x: 'yes' if x != 0 else
                                                           'no')

# feature is_renovated
df2['is_renovated'] = df2['yr_renovated'].apply( lambda x: 'yes' if x != 0 else
                                                           'no')

# feature 2000_or_1980

df2['2000_or_1980'] = df2['yr_built'].apply( lambda x: 2000 if ( x >= 2000 ) & ( x > 2010 ) else
                                                       1980 if ( x >= 1980 ) & ( x > 1990 ) else
                                                       'NA')

# feature seasons
df2['seasons'] = df2['month_day'].apply( lambda x: 'spring' if ( x >= '03-20' ) & ( x < '06-21' ) else
                                                   'summer' if ( x >= '06-21' ) & ( x < '09-22' ) else
                                                   'fall'   if ( x >= '09-22' ) & ( x < '12-21' ) else
                                                   'winter')
